In [1]:
!pip uninstall -y predictionguard

Found existing installation: predictionguard 1.4.2
Uninstalling predictionguard-1.4.2:
  Successfully uninstalled predictionguard-1.4.2


In [2]:
!pip install predictionguard

  Using cached predictionguard-1.4.2-py2.py3-none-any.whl (7.2 kB)


You should consider upgrading via the 'c:\users\hp\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import json
import re
import predictionguard as pg
import pandas as pd
from getpass import getpass

In [2]:
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

Enter your Prediction Guard access token: ········


In [3]:
test_df = pd.read_csv('test.csv')
test_df['Text'] = ''

with open('cleaned_transcripts.json', 'r') as f:
    transcripts = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_transcripts.json'

In [6]:
def extract_answer(transcript, question):
    prompts = {
        "What is the patient's name?": 
        f'''
        ### Instruction: Identify the name of the patient from the transcript. Just provide the name. No other text. 
        
        ### Input: {transcript}
        
        ### Response:
        ''',
        
        "What is the patient's age?": 
        f'''
        ### Instruction: Determine the patient's age in years from the transcript. Provide only the number. No years old.
        
        ### Input: {transcript}
        
        ### Response:
        ''',
        
        "What is the patient's condition?": 
        f'''
        ### Instruction: Identify the primary medical condition of the patient from the transcript. List the condition.
        
        ### Input: {transcript}
        
        ### Response:
        ''',
        
        "What symptoms is the patient experiencing?": 
        f'''
        ### Instruction: List the symptoms experienced by the patient as mentioned in the transcript. List the symptoms.
        
        ### Input: {transcript}
        
        ### Response:
        ''', 

        "What precautions did the doctor advise?": 
        f'''
        ### Instruction: List the precautions advised by the doctor in the transcript. List the precautions.
        
        ### Input: {transcript}
        
        ### Response:
        ''',
        
        "What drug did the doctor prescribe?": 
        f'''"
        ### Instruction: Identify the drug prescribed by the doctor in the transcript. List the drugs. If nothing is prescribed, say no prescription
        
        ### Input: {transcript}
        
        ### Response:
        '''
    }

    prompt = prompts[question].format(transcript=transcript)
    
    result = pg.Completion.create(
        model="Nous-Hermes-Llama2-13B",
        prompt=prompt
    )
    answer = result['choices'][0]['text']
    return answer.strip()
    except KeyError:
        return "Error: Question does not have the expected placeholder or is not in the prompts dictionary."
    except Exception as e:
        return f"Error: {str(e)}"


SyntaxError: invalid syntax (<ipython-input-6-0d4434b3e609>, line 66)

In [4]:
def extract_answer2(transcript, question):
    if question == "What is the patient's name?":
        instruction = "Identify the name of the patient from the transcript. Just provide the name. No other text."
    
    elif question == "What is the patient's age?":
        instruction = "Provide only the number indicating the patient's age from the transcript. No 'years old'."
    
    elif question == "What is the patient's condition?":
        instruction = "Identify the medical condition of the patient from the transcript. List only the condition."
    
    elif question == "What symptoms is the patient experiencing?":
        instruction = "Identify the symptoms experienced by the patient from the transcript. List the symptoms."
    
    elif question == "What precautions did the doctor advise?":
        instruction = "Identify the precautions advised by the doctor from the transcript. List the precautions."
    
    elif question == "What drug did the doctor prescribe?":
        instruction = "Identify the drug prescribed by the doctor from the transcript and list them. If nothing is prescribed, say 'no prescription'."
    
    else:
        instruction = question

    prompt = f'''
    ### Instruction: {instruction} 
    
    ### Input: {transcript}
    
    ### Response:
    '''.format(transcript=transcript)

    try:
        result = pg.Completion.create(
            model="Nous-Hermes-Llama2-13B",
            prompt=prompt
        )
        answer = result['choices'][0]['text']
        return answer.strip()
    
    except KeyError:
        return "Error: Question does not have the expected placeholder or is not in the prompts dictionary."

    except Exception as e:
        error_message = str(e)
        if "Could not make prediction" in error_message:
            print("Failed Prompt:", prompt)  
            return "Error: Could not make prediction."
        else:
            return f"Error: {error_message}"


In [5]:
part1 = test_df.iloc[0:3001].copy()
part2 = test_df.iloc[3001:6002].copy()
part3 = test_df.iloc[6002:9003].copy()
part4 = test_df.iloc[9003:12006].copy()

In [6]:
#test_subset = test_df.head(6).copy()
test_subset = part3.copy()
for idx, row in test_subset.iterrows():
    transcript = transcripts[str(row['Transcript'])]
    question = row['Question']
    answer = extract_answer2(transcript, question)
    test_subset.at[idx, 'Text'] = answer


Failed Prompt: 
    ### Instruction: Identify the symptoms experienced by the patient from the transcript. List the symptoms. 
    
    ### Input: خلال الزيارة، سمعت أنّي، الطبيب، أعراض جون بيرني من القيء والنقب والتعرق. بعد الفحص، تمّ تحديد أنّ السيد بيرني قدّ يعاني من نوبة قلبية. نصحت السيد بيرني بالحصول على الرعاية الطبية الفورية، لأنّه كان من الحيوي للحصول على سلامة بشرية. بسبب شدّة الحالة، لم يُشار إلى أيّ تدابير أو أدوية، وأوجب السيد بيرني على الذهاب مباشرة إلى الغرفة الطوارئ.
    
    ### Response:
    
Failed Prompt: 
    ### Instruction: Identify the symptoms experienced by the patient from the transcript. List the symptoms. 
    
    ### Input: During the visit, I, the doctor, assessed Alberta Studer and determined that she is suffering from paralysis due to a brain hemorrhage. She is experiencing symptoms such as vomiting, altered sensorium, and weakness of one body side. After evaluating her condition, I advised her to take precautions such as massage, eating healthy, and e

Failed Prompt: 
    ### Instruction: Identify the symptoms experienced by the patient from the transcript. List the symptoms. 
    
    ### Input: D: Good morning, Sharon. How can I help you today?

P: Hi, Doctor. I have been experiencing a loss of appetite, indigestion, and the passage of gases for some time now.

D: I see. Based on your symptoms, I believe you may have peptic ulcer disease. 

P: Is that common in older adults like me?

D: Peptic ulcer disease can occur at any age, but it is more common in older adults.

P: What should I do to manage my symptoms?

D: I would recommend avoiding fatty and spicy foods, consuming probiotic foods, eliminating milk, and limiting alcohol. Additionally, you may find it helpful to take antacids to reduce acid production in your stomach.

P: Are there any medications I should take?

D: At this time, I do not prescribe any medication for your condition. However, if your symptoms worsen or do not improve with lifestyle changes, we can discuss oth

Failed Prompt: 
    ### Instruction: Identify the precautions advised by the doctor from the transcript. List the precautions. 
    
    ### Input: D: Good mornin', Lydia. How ya feelin' today?
P: Not good, Doctor. I've been havin' chest pain and sweatin' a lot recently. 
D: I see. In addition to chest pain, have you been experiencing any other sympt$oms?
P: Yup, I've been vommiting a lot too. It's been really scaryy.
D: Based on your symtoms, it seems like you may be having a heart attack. I'll run some tests to confim this diagnosis. 
P: (panickked) A heart attack? Oh my gosh, is it seriouss??
D: Yes, it can be serious if not treated promptly. Don't worry, we'll take good care of you. 
P: (relievedd) Thank you, Doctor. What should I doo?
D: Just rest and try to stay calm. We'll get you admitted to the hospital right awayy. 
P: (anxiouss) Will I be okayy??
D: We'll do everything we can to take care of you. Let's get you to the hospital as soon as possibble. 
P: (resignedd) Okay, let's

Failed Prompt: 
    ### Instruction: Provide only the number indicating the patient's age from the transcript. No 'years old'. 
    
    ### Input: During the visit, I, as the doctor, determined that Jessie Marcum, a 71-year-old patient, has hepatitis B. The patient presented with symptoms such as dark urine, loss of appetite, and yellow urine. I advised the patient to consult the nearest hospital for further evaluation and treatment. I also recommended that the patient get vaccinated, eat healthily, and take medication as prescribed. Since the patient's condition was already diagnosed as hepatitis B, no medication was prescribed during this visit.
    
    ### Response:
    
Failed Prompt: 
    ### Instruction: Identify the medical condition of the patient from the transcript. List only the condition. 
    
    ### Input: During the visit, I, as the doctor, determined that Jessie Marcum, a 71-year-old patient, has hepatitis B. The patient presented with symptoms such as dark urine, lo

Failed Prompt: 
    ### Instruction: Identify the medical condition of the patient from the transcript. List only the condition. 
    
    ### Input: Durin-g to-da-y's visi_t, I, as the docto_r, diagno_sed Ray Lowe with diab_etes b_ased on his age, conditio_n, and s_ymptoms of irregu_lar suga_r level, bl_urred and disto_rted visio_n, and increased a_ppetite. I advis_ed him to take prec_autions by mainta_ining a bal_anced die_t, exer_cising regul_arly, and follow_ing up with subsequ_ent appo_intments for fur_ther eval_uation. I did not pres_cribe any medi_cation for him at th_is time.
    
    ### Response:
    
Failed Prompt: 
    ### Instruction: Identify the symptoms experienced by the patient from the transcript. List the symptoms. 
    
    ### Input: Durin-g to-da-y's visi_t, I, as the docto_r, diagno_sed Ray Lowe with diab_etes b_ased on his age, conditio_n, and s_ymptoms of irregu_lar suga_r level, bl_urred and disto_rted visio_n, and increased a_ppetite. I advis_ed him to take 

...

Failed Prompt: 
    ### Instruction: Identify the precautions advised by the doctor from the transcript. List the precautions. 
    
    ### Input: D: Good morning, Susan. How are you feeling today?
P: I don't feel well at all, Doctor. I have a severe headache, and my skin is turning yellow.
D: I see. Based on your symptoms, I think you have hepatitis E. 
P: What is hepatitis E, and how did I get it?
D: Hepatitis E is a liver infection caused by the hepatitis E virus. It is usually spread through contaminated food or water. 
P: Oh, I see. What should I do now?
D: First, you need to stop consuming alcohol as it can worsen the liver condition. Second, you need to rest and take medication to manage the symptoms. Finally, we need to monitor your condition closely to ensure that your liver function improves.
P: Okay, Doctor. I'll follow your advice. Thank you for your help.
D: You're welcome, Susan. Please come back for a follow-up appointment in a week.
    
    ### Response:
    
Failed P

Failed Prompt: 
    ### Instruction: Identify the symptoms experienced by the patient from the transcript. List the symptoms. 
    
    ### Input: Doctor: Hi Aaron, I understand you've been experiencing some symptoms. Can you tell me about them?

Patient: Yes, I've been having a persistent cough, feeling nauseous, and I've been having chest pains recently.

Doctor: I see. Based on your symptoms and your medical history, it seems like you might have GERD.

Patient: GERD? What's that?

Doctor: It stands for Gastroesophageal Reflux Disease. It's a condition where the acid in your stomach flows back into your esophagus, causing discomfort.

Patient: Oh, I see. So, what should I do about it?

Doctor: Well, there are a few things you can do to manage your symptoms. First, avoid eating fatty or spicy foods. Also, try to avoid lying down after eating. Maintaining a healthy weight and exercising regularly can also help.

Patient: Got it. And are there any medications you're prescribing for me?


Failed Prompt: 
    ### Instruction: Identify the name of the patient from the transcript. Just provide the name. No other text. 
    
    ### Input: During today's visit, I, the doctor, diagnosed Sylvia Johnston with hepatitis B, given her age of 38 and the symptoms she has been experiencing such as abdominal pain and receiving unsterile injections. I have advised her to consult the nearest hospital immediately for further treatment. Additionally, I have recommended that she gets vaccinated, eats healthily, and takes the prescribed medication. Since no medication is prescribed, it is crucial that she follows the preventative measures to avoid any further complications.
    
    ### Response:
    
Failed Prompt: 
    ### Instruction: Provide only the number indicating the patient's age from the transcript. No 'years old'. 
    
    ### Input: During today's visit, I, the doctor, diagnosed Sylvia Johnston with hepatitis B, given her age of 38 and the symptoms she has been experiencing s

Failed Prompt: 
    ### Instruction: Identify the medical condition of the patient from the transcript. List only the condition. 
    
    ### Input: D: صباح الخير، إدوارد. ماذا يمكنني القيام به لمساعدتك اليوم؟

P: مرحبا، الطبيب. لقد أشعر بشيء غريب مؤخرًا. كنت أواجه مشكلة في التوازن وأشعر بألم في الرأس كثيرًا. أيضًا أشعر بأنني لا أستطيع تركيز بنفس الكفاءة التي كنت أستطيعها سابقًا.

D: أنا أرى. منذ متى كنت تشعر بهذه الأعراض؟

P: لقد عندي هذه الأعراض منذ ما يقرب من أسبوعين.

D: هل أنت على علم بأي ظروف أساسية قد تسبب هذه الأعراض؟

P: حسنًا، ضغط الدم الخاص بي كان عاليًا منذ وقت، لذلك فكرت أن هذا يمكن أن يكون متعلقًا بذلك.

D: لقد كنت على حق. ضغط الدم الخاص بك يمكن أن يسبب هذه الأعراض. أوصيك بمحاولة بعض الإجراءات الاحتياطية مثل التأمل وحمامات الملح والتخفيف من الإجهاد والحصول على نوم مناسب. بالإضافة إلى ذلك، يكون من الأفضل مراقبة ضغط الدم الخاص بك بانتظام والاستشارة مع الطبيب إذا كانت الأعراض تزداد أو لم تحسن.

P: حسنًا، سأحاول هذه الأشياء. شكرًا، الطبيب.

D: مرحبًا بك، إدوارد. لا تنس أن تع

Failed Prompt: 
    ### Instruction: Identify the symptoms experienced by the patient from the transcript. List the symptoms. 
    
    ### Input: D: Hi Jeffrey, thanks for coming in. Can you tell me about your sy~ptoms?

P: Hi, yes, I've been having a lot of headahces lately. Sometimes they're really bad and I feel like something is pressing down on my head. I also feel sad and have tr~ouble concentrating. My vision sometimes gets blurry too.

D: Based on your sympt~oms, it sounds like you might be experiencing migraines. Do you have any famil~y history of migraines?

P: Yes, my mom has them too.

D: Okay, let's talk about some prec@autions you can take to help manage your migraines. Have you tried medit@ation or mindfu~lness techniques?

P: No, I haven't. But I'm open to trying anything that might help.

D: Great, I think that could be really helpful for you. Reducing str~~ess and practicing mindfulness can help reduce the frequency of your migraines. Also, since you mentioned that s

In [48]:
test_subset

,Id,Transcript,Question,Text
0,587d0feb-5780-43e1-9595-e19d4b31dc07,2055,What is the patient's name?,Mr. Don Hicks
1,263e8884-e8ba-4266-bb0c-85271419a0b3,2055,What is the patient's age?,81
2,74c68eca-61b2-49d0-9b1c-0f6f886b04ff,2055,What is the patient's condition?,Fungal infection.
3,8572ab5d-f20a-4de5-ab44-f42b07e45a00,2055,What symptoms is the patient experiencing?,1. Fungal infection\n 2. Dischromic patches...
4,f5c92075-ef05-4fbf-a7a0-aa86c586ff02,2055,What precautions did the doctor advise?,1. Bathe twice a day\n2. Use detol or neem in ...
5,03406fb0-e67d-4614-a745-ed02c7ac6c46,2055,What drug did the doctor prescribe?,No prescription.


In [7]:
test_subset.to_csv('results_subset.csv', index=False)

In [ ]:
for idx, row in part1.iterrows():
    transcript = transcripts[str(row['Transcript'])]
    question = row['Question']
    answer = extract_answer2(transcript, question)
    part1.at[idx, 'Text'] = answer